In [82]:
import pandas as pd 

In [83]:
order_report = pd.read_csv("../csv/Order Report.csv")
sku_master = pd.read_csv("../csv/SKU Master.csv")
pincode_mapping = pd.read_csv("../csv/pincodes.csv")
courier_invoice = pd.read_csv("../csv/Invoice.csv")
courier_company_rates = pd.read_csv("../csv/Courier Company - Rates.csv")


In [84]:
order_report = order_report.drop(columns=["Unnamed: 3","Unnamed: 4"])
sku_master = sku_master.drop(columns=["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"])
pincode_mapping = pincode_mapping.drop(columns=["Unnamed: 3", "Unnamed: 4"])

In [85]:
merged_data = pd.merge(order_report, sku_master, on="SKU")
merged_data = merged_data.rename(columns={"ExternOrderNo":"Order ID"})

In [86]:
abc_courier = pincode_mapping.drop_duplicates(subset=["Customer Pincode"])
courier_abc = courier_invoice[['Order ID', 'Customer Pincode','Type of Shipment']]
pincodes = courier_abc.merge(abc_courier, on="Customer Pincode")

In [88]:
merged2 = merged_data.merge(pincodes, on='Order ID')
merged2['Weights (Kgs)'] = merged2['Weight (g)'] / 1000

In [89]:
def weight_slab(weight):
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) + 1
    else: 
        return int(weight) + 0.5

In [98]:
merged2['Weight Slab (KG)'] = merged2['Weights (Kgs)'].apply(weight_slab)

In [99]:
courier_invoice['Weight Slab Charged by Courier Company']=(courier_invoice['Charged Weight']).apply(weight_slab)

In [100]:
courier_invoice = courier_invoice.rename(columns={'Zone': 'Delivery Zone Charged by Courier Company'})
merged2 = merged2.rename(columns={'Zone': 'Delivery Zone As Per ABC'})
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per ABC'})

In [101]:
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per ABC'})

In [102]:
merged2

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per ABC,Weights (Kgs),Weight (Kgs),Weight Slab As Per ABC
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,0.5
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,0.5
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,0.5
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,0.5
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...
396,2001806251,8904223819161,1.0,115,743263,Forward charges,121003,d,0.115,0.5,0.5
397,2001806251,8904223819260,1.0,130,743263,Forward charges,121003,d,0.130,0.5,0.5
398,2001806233,8904223819161,1.0,115,263139,Forward charges,121003,b,0.115,0.5,0.5
399,2001806233,8904223819260,1.0,130,263139,Forward charges,121003,b,0.130,0.5,0.5


In [103]:
total_expected_charge = []
for _, row in merged2.iterrows():
    fwd_category = 'fwd_' + row['Delivery Zone As Per ABC']
    fwd_fixed = courier_company_rates.at[0, fwd_category + '_fixed']
    fwd_additional = courier_company_rates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery Zone As Per ABC']
    rto_fixed = courier_company_rates.at[0, rto_category + '_fixed']
    rto_additional = courier_company_rates.at[0, rto_category + '_additional']

    weight_slab = row['Weight Slab As Per ABC']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)

In [104]:
merged2['Expected Charge as per ABC'] = total_expected_charge

In [105]:
merged2

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery Zone As Per ABC,Weights (Kgs),Weight (Kgs),Weight Slab As Per ABC,Expected Charge as per ABC
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,0.5,56.6
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,0.5,56.6
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,0.5,56.6
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,0.5,56.6
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,0.5,56.6
...,...,...,...,...,...,...,...,...,...,...,...,...
396,2001806251,8904223819161,1.0,115,743263,Forward charges,121003,d,0.115,0.5,0.5,45.4
397,2001806251,8904223819260,1.0,130,743263,Forward charges,121003,d,0.130,0.5,0.5,45.4
398,2001806233,8904223819161,1.0,115,263139,Forward charges,121003,b,0.115,0.5,0.5,33.0
399,2001806233,8904223819260,1.0,130,263139,Forward charges,121003,b,0.130,0.5,0.5,33.0


In [107]:
merged_output = merged2.merge(courier_invoice, on='Order ID')
merged_output

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode_x,Type of Shipment_x,Warehouse Pincode_x,Delivery Zone As Per ABC,Weights (Kgs),Weight (Kgs),Weight Slab As Per ABC,Expected Charge as per ABC,AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode_y,Delivery Zone Charged by Courier Company,Type of Shipment_y,Billing Amount (Rs.),Weight Slab Charged by Courier Company
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,0.5,56.6,1091122418320,1.60,121003,173213,b,Forward charges,117.9,2.0
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,0.5,56.6,1091122418320,1.60,121003,173213,b,Forward charges,117.9,2.0
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,0.5,56.6,1091122418320,1.60,121003,173213,b,Forward charges,117.9,2.0
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,0.5,56.6,1091122418320,1.60,121003,173213,b,Forward charges,117.9,2.0
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,0.5,56.6,1091122418320,1.60,121003,173213,b,Forward charges,117.9,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,2001806251,8904223819161,1.0,115,743263,Forward charges,121003,d,0.115,0.5,0.5,45.4,1091117222146,1.27,121003,743263,d,Forward charges,135.0,1.5
397,2001806251,8904223819260,1.0,130,743263,Forward charges,121003,d,0.130,0.5,0.5,45.4,1091117222146,1.27,121003,743263,d,Forward charges,135.0,1.5
398,2001806233,8904223819161,1.0,115,263139,Forward charges,121003,b,0.115,0.5,0.5,33.0,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0
399,2001806233,8904223819260,1.0,130,263139,Forward charges,121003,b,0.130,0.5,0.5,33.0,1091117222135,0.78,121003,263139,b,Forward charges,61.3,1.0


In [109]:
df_diff = merged_output.copy()
df_diff['Difference (Rs.)'] = df_diff['Billing Amount (Rs.)'] - df_diff['Expected Charge as per ABC']

In [110]:
df_new = df_diff[['Order ID', 'Difference (Rs.)', 'Expected Charge as per ABC']]

In [111]:
df_new

,Order ID,Difference (Rs.),Expected Charge as per ABC
0,2001827036,61.3,56.6
1,2001827036,61.3,56.6
2,2001827036,61.3,56.6
3,2001827036,61.3,56.6
4,2001827036,61.3,56.6
...,...,...,...
396,2001806251,89.6,45.4
397,2001806251,89.6,45.4
398,2001806233,28.3,33.0
399,2001806233,28.3,33.0


In [112]:
# Calculate the total orders in each category
total_correctly_charged = len(df_new[df_new['Difference (Rs.)'] == 0])
total_overcharged = len(df_new[df_new['Difference (Rs.)'] > 0])
total_undercharged = len(df_new[df_new['Difference (Rs.)'] < 0])

In [113]:
# Calculate the total amount in each category
amount_overcharged = abs(df_new[df_new['Difference (Rs.)'] > 0]['Difference (Rs.)'].sum())
amount_undercharged = df_new[df_new['Difference (Rs.)'] < 0]['Difference (Rs.)'].sum()
amount_correctly_charged = df_new[df_new['Difference (Rs.)'] == 0]['Expected Charge as per ABC'].sum()

In [114]:
# Create a new DataFrame for the summary
summary_data = {'Description': ['Total Orders where ABC has been correctly charged',
                                'Total Orders where ABC has been overcharged',
                                'Total Orders where ABC has been undercharged'],
                'Count': [total_correctly_charged, total_overcharged, total_undercharged],
                'Amount (Rs.)': [amount_correctly_charged, amount_overcharged, amount_undercharged]}

df_summary = pd.DataFrame(summary_data)

print(df_summary)

                                         Description  Count  Amount (Rs.)
0  Total Orders where ABC has been correctly charged     12         507.6
1        Total Orders where ABC has been overcharged    382       33750.5
2       Total Orders where ABC has been undercharged      7        -165.2


In [115]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Pie(labels=df_summary['Description'],
                            values=df_summary['Count'],
                            textinfo='label+percent',
                            hole=0.4))
fig.update_layout(title='Proportion')

fig.show()